In [ ]:
# default_exp resimulation

# Match resimulation

> Simulating match outcomes based on the xG of individual shots

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import collections
import itertools

import numpy as np

In [ ]:
#export


def poisson_binomial_pmf(probs, xs, n=int(10000)):
    """
    Simulate events by Monte-Carlo to get probability estimates of
    exactly x events occuring.
    """
    sim = collections.Counter()
    for _ in range(n):
        goals = sum(np.random.binomial(1, p) for p in probs)
        sim[goals] += (1/n)
    return [sim[x] for x in xs]


def resimulate_match(shots, up_to=26, min_xg=0.0001, **kwargs):
    """
    Resimulate a match based on xG. Takes a list of maps, where each map
    represents a shot has and has 'is_home' (bool) and 'xg' (float) keys.
    """

    # Prevent potential underflow
    home_xgs = [max(s['xg'], min_xg) for s in shots if s['is_home']]
    away_xgs = [max(s['xg'], min_xg) for s in shots if not s['is_home']]

    home_scores = list(range(min(len(home_xgs) + 1, up_to)))
    away_scores = list(range(min(len(away_xgs) + 1, up_to)))

    home_probs = dict(zip(home_scores, poisson_binomial_pmf(home_xgs, home_scores)))
    away_probs = dict(zip(away_scores, poisson_binomial_pmf(away_xgs, away_scores)))

    scores = []
    for h, a in itertools.product(range(up_to), repeat=2):
        home_prob = home_probs.get(h, 0)
        away_prob = away_probs.get(a, 0)
        scores.append({
            'home_goals': h,
            'away_goals': a,
            'home_probability': home_prob,
            'away_probability': away_prob,
            'probability': home_prob*away_prob,
            **kwargs
        })

    # Keep everything up to 4-4; filter out P == 0 results above that
    return [
        s for s in scores 
        if s['probability'] > 0 
        or (s['home_goals'] < 5 and s['away_goals'] < 5)
    ]
